In [1]:
from sklearn.cluster import KMeans
import pandas as pd 
from sqlite3 import connect
from sklearn import preprocessing

conn = connect('/content/drive/MyDrive/geotesouro/data/data.db')
agg_municipios = pd.read_sql('SELECT * FROM agg_municipios', conn)

In [2]:
X = agg_municipios.drop(["codigo_ibge", "siafi_id", "nome", "uf_sigla"], 1)

le = preprocessing.LabelEncoder()
le.fit(agg_municipios["fuso_horario"])
X["fuso_horario"] = le.transform(agg_municipios["fuso_horario"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
sims = pd.DataFrame(columns = ["codigo_ibge", "siafi_id", "nome", "cod_mun_sim_rank1", "cod_mun_sim_rank2", "cod_mun_sim_rank3", "cod_mun_sim_rank4", "cod_mun_sim_rank5", 
                               "cod_mun_sim_rank6", "cod_mun_sim_rank7", "cod_mun_sim_rank8", "cod_mun_sim_rank9",  "cod_mun_sim_rank10"])

sims[["codigo_ibge", "siafi_id", "nome"]] = agg_municipios[["codigo_ibge", "siafi_id", "nome"]]

In [3]:
sims = pd.read_sql('SELECT * FROM similaridades', conn)

In [4]:
sims[sims["cod_mun_sim_rank1"].isnull()]

,codigo_ibge,siafi_id,nome,cod_mun_sim_rank1,cod_mun_sim_rank2,cod_mun_sim_rank3,cod_mun_sim_rank4,cod_mun_sim_rank5,cod_mun_sim_rank6,cod_mun_sim_rank7,cod_mun_sim_rank8,cod_mun_sim_rank9,cod_mun_sim_rank10
644,4103057,7981,Boa Vista da Aparecida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,4302154,5981,Boa Vista das Missões,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,4302204,8543,Boa Vista do Buricá,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,4302220,1124,Boa Vista do Cadeado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,2101970,126,Boa Vista do Gurupi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,2933604,3971,Xique-Xique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5566,2517407,542,Zabelê,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5567,3557154,2973,Zacarias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5568,2114007,1287,Zé Doca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

sims = pd.read_sql('SELECT * FROM similaridades', conn)
list_index = X.index.to_list()
X = X.fillna(-9999999999)

for m in tqdm(sims[sims["cod_mun_sim_rank1"].isnull()].index):
  rank = pd.DataFrame(columns = ["codigo_ibge", "nome", "valor"])

  for i in list_index:
    r = pd.DataFrame(columns = ["codigo_ibge", "nome", "valor"])
    r = r.append({"codigo_ibge": agg_municipios.loc[i]["codigo_ibge"], "nome": agg_municipios.loc[i]["nome"], "valor": cosine_similarity(X.loc[m].values.reshape(1, -1), X.loc[i].values.reshape(1, -1))[0][0]}, ignore_index=True)
    rank = pd.concat([rank, r])

  rank = rank.sort_values(by='valor', ascending=False)[1:11].reset_index(drop=True)

  for i in rank["valor"].index:
    sims[f"cod_mun_sim_rank{i+1}"].loc[m] = rank["codigo_ibge"].loc[i]
  
  sims.to_sql("similaridades", con = conn, if_exists = "replace", index=False)


  0%|          | 0/4926 [00:09<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
sims

,codigo_ibge,siafi_id,nome,cod_mun_sim_rank1,cod_mun_sim_rank2,cod_mun_sim_rank3,cod_mun_sim_rank4,cod_mun_sim_rank5,cod_mun_sim_rank6,cod_mun_sim_rank7,cod_mun_sim_rank8,cod_mun_sim_rank9,cod_mun_sim_rank10
0,5200050,1050,Abadia de Goiás,3140159.0,2108454.0,4201950.0,4301636.0,4300877.0,2106326.0,3302452.0,3204955.0,2100436.0,4210035.0
1,3100104,4001,Abadia dos Dourados,3502705.0,4125902.0,1716505.0,4312302.0,2917201.0,4113452.0,3111101.0,3200300.0,2931004.0,4310306.0
2,5200100,9201,Abadiânia,3112307.0,3138609.0,4111308.0,4114104.0,3168002.0,2700300.0,2414209.0,4206009.0,5100201.0,5002209.0
3,3100203,4003,Abaeté,4319406.0,4102406.0,5211206.0,3538907.0,3522406.0,4314704.0,3101904.0,4116208.0,3152006.0,4302204.0
4,1500107,401,Abaetetuba,2202307.0,2922730.0,2906600.0,2314003.0,2101707.0,2211308.0,2600302.0,2205706.0,2919405.0,2404200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,2933604,3971,Xique-Xique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5566,2517407,542,Zabelê,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5567,3557154,2973,Zacarias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5568,2114007,1287,Zé Doca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
